In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -qqq -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -qqq datasets bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

torch.cuda.set_device('cuda:0')

# we use this sharded model because the orignal model is too big to run in colab
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
torch.cuda.set_device('cuda:0')
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)

model.config.use_cache = False
# load the tokenizer
torch.cuda.set_device('cuda:0')
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
#add adapter
from peft import PeftModel
adapters = '/content/drive/MyDrive/AI_dataset/felcon7B/fine_tuned_model/'
model = PeftModel.from_pretrained(model, adapters)

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
Question: : "There is car in front of us, there is a bicycle crossing the road and we want to turn leeft."
Answer:
""".strip()

torch.cuda.set_device('cuda:0')
encoding = tokenizer(prompt, return_tensors="pt")
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Question: : "There is car in front of us, there is a bicycle crossing the road and we want to turn leeft."
Answer: "Turn on the left turn signal, wait for the car in front to move, complete the bicycle crossing the road, then turn left."
CPU times: user 3.07 s, sys: 0 ns, total: 3.07 s
Wall time: 3.66 s
